In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [2]:
dealing_data=pd.read_csv("dealing_data.csv",index_col=0,parse_dates=True)

In [3]:
opaque=pd.read_csv("opaque.csv")
opaque = opaque[np.isfinite(opaque['gind'])]
opaque_tic=opaque.set_index('tic')
opaque_gic=opaque.set_index('gind')
opaque_gic.index=opaque_gic.index.dropna().astype(int).astype(str)

In [4]:
opaque_year=pd.read_csv("opaque_year.csv")
opaque_year_tic=opaque_year.set_index("tic")

In [7]:
# count=0
# enough_name=[]
# for name in list(set(opaque_year_tic.index))[1:]:
#     if abs(opaque_year_tic.loc[name].shape[0]-opaque_year_tic.loc[name].dropna().shape[0])<2 and opaque_year_tic.loc[name].dropna().shape[0]>4:
#         enough_name.append(name)
#     else:
#         print(name)
# with open('enough_name.txt', 'w') as f:
#     for item in enough_name:
#         f.write("%s\n" % item)

In [5]:
enough_name=open("enough_name.txt", "r").read().split("\n")[:-1]

In [6]:
class_industry=pd.DataFrame(columns=["gic"])
for name in enough_name:
    try:
        class_industry.loc[name]=opaque_year_tic["gind"].loc[name].iloc[0]
    except:
        class_industry.loc[name]=opaque_year_tic["gind"].loc[name]

In [7]:
class_industry=class_industry.dropna().astype(int).astype(str)
enough_name=list(class_industry.index)

In [8]:
class_industry=class_industry.reset_index().set_index("gic")

In [9]:
all_industry=class_industry.index
industry=[]
for i in all_industry:
    if i not in industry:
        industry.append(i)

In [10]:
remove_index=["401010","401020","402010","402020","402030","402040","403010","404010","404030","404020","551010","551020","551030","551040","551050","601010","601020"]

In [11]:
for index in remove_index:
    industry.remove(index)

In [12]:
industry

['351020',
 '252010',
 '201070',
 '251010',
 '201010',
 '151030',
 '451020',
 '451010',
 '151040',
 '352010',
 '452010',
 '502030',
 '451030',
 '201030',
 '253020',
 '303020',
 '202010',
 '351010',
 '101020',
 '252020',
 '201040',
 '352020',
 '452020',
 '452030',
 '453010',
 '254010',
 '151010',
 '253010',
 '203020',
 '203040',
 '501010',
 '255010',
 '201060',
 '201020',
 '252030',
 '202020',
 '101010',
 '255040',
 '302020',
 '501020',
 '151050',
 '351030',
 '255020',
 '352030',
 '502010',
 '302010',
 '203050',
 '203010',
 '452040',
 '301010',
 '303010',
 '151020',
 '203030',
 '502020',
 '452050',
 '302030',
 '201050',
 '251020',
 '255030']

In [78]:
"""
k=0
company_name=class_industry.loc[industry[k]].values.reshape(-1).tolist()
industry_temp_memb=opaque_year_tic.loc[company_name]
industry_temp_memb=industry_temp_memb.dropna(how='any')
industry_company=pd.DataFrame()
for i in company_name:
    try:
        temp_date=pd.to_datetime(industry_temp_memb.loc[i]["datadate"])
        shift_date=(temp_date-temp_date.shift(1)).dropna()
        if ((shift_date.apply(lambda x: x.days)<400)/1).sum()==len(shift_date) and ((industry_temp_memb.loc[i]["at"]>0)/1).sum() == industry_temp_memb.loc[i].shape[0]:
            industry_company=pd.concat([industry_company,industry_temp_memb.loc[i][["datadate","at","revt","xidoc","ibc","oancf","recch","ppent"]]])
    except:
        continue
filter_name=list(set(industry_company.index))
input_temp=pd.DataFrame(columns=["x1","x2","x3","x4","y","index_tic","index_time"])
for name in filter_name:
    caculate_temp=industry_company.loc[name].set_index("datadate")
    time_temp=pd.to_datetime(caculate_temp.index)
    caculate_temp.index=time_temp
    for j in range(1,len(time_temp)):
        x1=1/caculate_temp["at"][j-1]
        x2=(caculate_temp["revt"][j]-caculate_temp["revt"][j-1])/caculate_temp["at"][j-1]
        x3=caculate_temp["ppent"][j]/caculate_temp["at"][j-1]
        x4=industry_company["recch"][j]/caculate_temp["at"][j-1]
        y=(caculate_temp["ibc"][j]-(caculate_temp["oancf"][j]-caculate_temp["xidoc"][j]))/caculate_temp["at"][j-1]
        input_temp=input_temp.append({"x1":x1,"x2":x2,"x3":x3,"x4":x4,"y":y,"index_tic":name,"index_time":time_temp[j]},ignore_index=True)
input_temp=input_temp.set_index(["index_tic","index_time"])
model = sm.OLS(input_temp["y"],input_temp[["x1","x2","x3"]])
results = model.fit()
params=results.params
dissacc=input_temp["y"]-(params[0]*input_temp["x1"]+params[1]*(input_temp["x2"]-input_temp["x4"])+params[2]*input_temp["x3"])
for name in set(dissacc.index.get_level_values(0)):
    temp_stored=abs(dissacc.loc[name]).rolling(window=3).mean().dropna().reset_index()
    temp_stored.index=[name]*len(temp_stored)
    temp_stored.columns=["time","opaque"]
"""

In [83]:
opaque_index_year=pd.DataFrame(columns=["time","opaque"])
for k in range(len(industry)):
#for k in range(2):
    print(k)
    company_name=class_industry.loc[industry[k]].values.reshape(-1).tolist()
    industry_temp_memb=opaque_year_tic.loc[company_name]
    industry_temp_memb=industry_temp_memb.dropna(how='any')
    industry_company=pd.DataFrame()
    for i in company_name:
        try:
            temp_date=pd.to_datetime(industry_temp_memb.loc[i]["datadate"])
            shift_date=(temp_date-temp_date.shift(1)).dropna()
            if ((shift_date.apply(lambda x: x.days)<400)/1).sum()==len(shift_date) and ((industry_temp_memb.loc[i]["at"]>0)/1).sum() == industry_temp_memb.loc[i].shape[0]:
                industry_company=pd.concat([industry_company,industry_temp_memb.loc[i][["datadate","at","revt","xidoc","ibc","oancf","recch","ppent"]]])
        except:
            continue
    filter_name=list(set(industry_company.index))
    input_temp=pd.DataFrame(columns=["x1","x2","x3","x4","y","index_tic","index_time"])
    for name in filter_name:
        caculate_temp=industry_company.loc[name].set_index("datadate")
        time_temp=pd.to_datetime(caculate_temp.index)
        caculate_temp.index=time_temp
        for j in range(1,len(time_temp)):
            x1=1/caculate_temp["at"][j-1]
            x2=(caculate_temp["revt"][j]-caculate_temp["revt"][j-1])/caculate_temp["at"][j-1]
            x3=caculate_temp["ppent"][j]/caculate_temp["at"][j-1]
            x4=industry_company["recch"][j]/caculate_temp["at"][j-1]
            y=(caculate_temp["ibc"][j]-(caculate_temp["oancf"][j]-caculate_temp["xidoc"][j]))/caculate_temp["at"][j-1]
            input_temp=input_temp.append({"x1":x1,"x2":x2,"x3":x3,"x4":x4,"y":y,"index_tic":name,"index_time":time_temp[j]},ignore_index=True)
    input_temp=input_temp.set_index(["index_tic","index_time"])
    model = sm.OLS(input_temp["y"],input_temp[["x1","x2","x3"]])
    results = model.fit()
    params=results.params
    dissacc=input_temp["y"]-(params[0]*input_temp["x1"]+params[1]*(input_temp["x2"]-input_temp["x4"])+params[2]*input_temp["x3"])
    for name in set(dissacc.index.get_level_values(0)):
        temp_stored=abs(dissacc.loc[name]).rolling(window=3).mean().dropna().reset_index()
        temp_stored.index=[name]*len(temp_stored)
        temp_stored.columns=["time","opaque"]
        opaque_index_year=pd.concat([opaque_index_year,temp_stored])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


In [64]:
#opaque_index_year.to_csv("opaque_index_year.csv")

In [62]:
# for name in class_industry.loc[["404010","404030","404020"]].values.reshape(-1).tolist():
#     try:
#         opaque_index_year=opaque_index_year.drop(name)
#     except:
#         continue

In [13]:
opaque_index_year=pd.read_csv("opaque_index_year.csv",index_col=0)

In [155]:
"""
count=0
opaque_index=pd.DataFrame(columns=["time","opaque","datacqtr"])
k=0
company_name=class_industry.loc[industry[k]].values.reshape(-1).tolist()
industry_temp_memb=opaque_tic.loc[company_name]
industry_company=pd.DataFrame()
for i in company_name:
    now=industry_temp_memb.loc[i]
    delete_num=now.fillna(method='ffill').shape[0]-now.fillna(method='ffill').dropna().shape[0]
    if abs(now.iloc[delete_num:,:].dropna().shape[0]-now.iloc[delete_num:,:].shape[0])<3:
        try:
            now=now.fillna(method='ffill').dropna()
            temp_date=pd.to_datetime(now["datadate"])
            shift_date=(temp_date-temp_date.shift(1)).dropna()
            if ((shift_date.apply(lambda x: x.days)<150)/1).sum()==len(shift_date) and len(shift_date)>12 and ((now["atq"]>0)/1).sum() == now.shape[0]:
                industry_company=pd.concat([industry_company,now[["datadate","atq","revtq","xidocy","ibcy","oancfy","recchy","ppentq","datacqtr"]]])
        except:
            continue
filter_name=list(set(industry_company.index))
input_temp=pd.DataFrame(columns=["x1","x2","x3","x4","y","index_tic","index_time","index_quarter"])
for name in filter_name:
    caculate_temp=industry_company.loc[name].set_index("datadate")
    time_temp=pd.to_datetime(caculate_temp.index)
    caculate_temp.index=time_temp
    for j in range(1,len(time_temp)):
        x1=1/caculate_temp["atq"][j-1]
        x2=(caculate_temp["revtq"][j]-caculate_temp["revtq"][j-1])/caculate_temp["atq"][j-1]
        x3=caculate_temp["ppentq"][j]/caculate_temp["atq"][j-1]
        x4=industry_company["recchy"][j]/caculate_temp["atq"][j-1]
        index_quarter=caculate_temp["datacqtr"][j]
        y=(caculate_temp["ibcy"][j]-(caculate_temp["oancfy"][j]-caculate_temp["xidocy"][j]))/caculate_temp["atq"][j-1]
        input_temp=input_temp.append({"x1":x1,"x2":x2,"x3":x3,"x4":x4,"y":y,"index_tic":name,"index_time":time_temp[j],"index_quarter":index_quarter},ignore_index=True)
input_temp=input_temp.set_index(["index_tic","index_time"])
model = sm.OLS(input_temp["y"],input_temp[["x1","x2","x3"]])
results = model.fit()
params=results.params
dissacc=input_temp["y"]-(params[0]*input_temp["x1"]+params[1]*(input_temp["x2"]-input_temp["x4"])+params[2]*input_temp["x3"])
for name in set(dissacc.index.get_level_values(0)):
    temp_stored=abs(dissacc.loc[name]).rolling(window=12).mean().dropna().reset_index()
    temp_stored.index=[name]*len(temp_stored)
    temp_stored.columns=["time","opaque"]
    temp_stored["datacqtr"]=input_temp.loc[name]["index_quarter"][11:].values
    opaque_index=pd.concat([opaque_index,temp_stored])
"""

In [172]:
# opaque_index=pd.DataFrame(columns=["time","opaque","datacqtr"])
# for k in range(len(industry)):
# #for k in range(2):
#     print(k)
#     company_name=class_industry.loc[industry[k]].values.reshape(-1).tolist()
#     industry_temp_memb=opaque_tic.loc[company_name]
#     industry_company=pd.DataFrame()
#     for i in company_name:
#         now=industry_temp_memb.loc[i]
#         try:
#             delete_num=now.fillna(method='ffill').shape[0]-now.fillna(method='ffill').dropna().shape[0]
#             if abs(now.iloc[delete_num:,:].dropna().shape[0]-now.iloc[delete_num:,:].shape[0])<3:
#                     now=now.fillna(method='ffill').dropna()
#                     temp_date=pd.to_datetime(now["datadate"])
#                     shift_date=(temp_date-temp_date.shift(1)).dropna()
#                     if ((shift_date.apply(lambda x: x.days)<150)/1).sum()==len(shift_date) and len(shift_date)>12 and ((now["atq"]>0)/1).sum() == now.shape[0]:
#                         industry_company=pd.concat([industry_company,now[["datadate","atq","revtq","xidocy","ibcy","oancfy","recchy","ppentq","datacqtr"]]])
#         except:
#             continue
#     filter_name=list(set(industry_company.index))
#     input_temp=pd.DataFrame(columns=["x1","x2","x3","x4","y","index_tic","index_time","index_quarter"])
#     for name in filter_name:
#         caculate_temp=industry_company.loc[name].set_index("datadate")
#         time_temp=pd.to_datetime(caculate_temp.index)
#         caculate_temp.index=time_temp
#         for j in range(1,len(time_temp)):
#             x1=1/caculate_temp["atq"][j-1]
#             x2=(caculate_temp["revtq"][j]-caculate_temp["revtq"][j-1])/caculate_temp["atq"][j-1]
#             x3=caculate_temp["ppentq"][j]/caculate_temp["atq"][j-1]
#             if time_temp[j].month<5:
#                     x4=caculate_temp["recchy"][j]/caculate_temp["atq"][j-1]
#             else:
#                 x4=(caculate_temp["recchy"][j]-caculate_temp["recchy"][j-1])/caculate_temp["atq"][j-1]
#             index_quarter=caculate_temp["datacqtr"][j]
#             y=(caculate_temp["ibcy"][j]-(caculate_temp["oancfy"][j]-caculate_temp["xidocy"][j]))/caculate_temp["atq"][j-1]
#             input_temp=input_temp.append({"x1":x1,"x2":x2,"x3":x3,"x4":x4,"y":y,"index_tic":name,"index_time":time_temp[j],"index_quarter":index_quarter},ignore_index=True)
#     input_temp=input_temp.set_index(["index_tic","index_time"])
#     model = sm.OLS(input_temp["y"],input_temp[["x1","x2","x3"]])
#     results = model.fit()
#     params=results.params
#     dissacc=input_temp["y"]-(params[0]*input_temp["x1"]+params[1]*(input_temp["x2"]-input_temp["x4"])+params[2]*input_temp["x3"])
#     for name in set(dissacc.index.get_level_values(0)):
#         temp_stored=abs(dissacc.loc[name]).rolling(window=12).mean().dropna().reset_index()
#         temp_stored.index=[name]*len(temp_stored)
#         temp_stored.columns=["time","opaque"]
#         temp_stored["datacqtr"]=input_temp.loc[name]["index_quarter"][11:].values
#         opaque_index=pd.concat([opaque_index,temp_stored])

0
1
2
3
4
5


/shared_home/willie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


In [175]:
#opaque_index.to_csv("opaque_index_quarter.csv")

In [185]:
#opaque_index_quarter=pd.read_csv("opaque_index_quarter.csv",index_col=0)

In [186]:
# year_index=list(set(opaque_index_year.index))
# quarter_index=list(set(opaque_index_quarter.index))

In [197]:
# count=0
# for name in quarter_index:
#     if name not in year_index:
#         count+=1

In [103]:
cut_data=pd.read_csv("cut_data_all_QK.csv",index_col=0,parse_dates=["date"])

In [104]:
cut_data.loc["AAPL"]

,date,adjust_starttime,adjust_endtime,return,bad,url,QK
company,,,,,,,
AAPL,1994-12-13,1994-12-13,1994-12-30,0.993610,0.0,/Archives/edgar/data/320193/0000320193-94-0000...,K
AAPL,1994-12-30,1994-12-30,1995-03-31,0.973724,0.0,/Archives/edgar/data/320193/0000320193-95-0000...,Q
AAPL,1995-03-31,1995-03-31,1995-06-30,1.330258,0.0,/Archives/edgar/data/320193/0000320193-95-0000...,Q
AAPL,1995-12-19,1995-12-19,1996-02-13,0.918606,0.0,/Archives/edgar/data/320193/0000320193-95-0000...,K
AAPL,1995-12-29,1995-12-29,1996-03-29,0.803963,1.0,/Archives/edgar/data/320193/0000320193-96-0000...,Q
AAPL,1996-03-29,1996-03-29,1996-06-28,0.937823,0.0,/Archives/edgar/data/320193/0000320193-96-0000...,Q
AAPL,1996-12-19,1996-12-19,1997-02-11,0.664865,1.0,/Archives/edgar/data/320193/0000320193-96-0000...,K
AAPL,1996-12-27,1996-12-27,1997-03-28,0.791664,1.0,/Archives/edgar/data/320193/0000320193-97-0000...,Q
AAPL,1997-03-28,1997-03-28,1997-06-27,0.824501,1.0,/Archives/edgar/data/320193/0000320193-97-0000...,Q


In [105]:
cut_data["opaque"]=np.zeros(cut_data.shape[0])

In [106]:
company_inboth=[]
for name in list(set(opaque_index_year.index)):
    if name in list(set(cut_data.index)):
        company_inboth.append(name)

In [107]:
len(company_inboth)

3033

In [108]:
cut_data

,date,adjust_starttime,adjust_endtime,return,bad,url,QK,opaque
company,,,,,,,,
A,2000-03-15,2000-03-15,2000-06-12,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q,0.0
A,2000-06-12,2000-06-12,2000-09-01,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q,0.0
A,2000-09-01,2000-09-01,2001-01-17,0.893940,0.0,/Archives/edgar/data/1090872/00009120570004000...,Q,0.0
A,2001-01-17,2001-01-17,2001-03-19,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K,0.0
A,2001-03-19,2001-03-19,2001-06-14,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q,0.0
A,2001-06-14,2001-06-14,2001-09-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,Q,0.0
A,2001-09-14,2001-09-14,2002-01-22,1.462535,0.0,/Archives/edgar/data/1090872/00010958110150492...,Q,0.0
A,2002-01-22,2002-01-22,2002-03-06,1.034088,0.0,/Archives/edgar/data/1090872/00008916180200018...,K,0.0
A,2002-03-06,2002-03-06,2002-06-05,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...,Q,0.0


In [109]:
name="A"
public_time=pd.to_datetime(cut_data.loc[name].set_index("QK").loc["K"]["date"])
report_time=pd.to_datetime(opaque_index_year.loc[name]["time"])
QK = cut_data.loc[name]['QK'].values
op = cut_data.loc[name]['opaque'].values
print(name)
record=[]
for j in range(len(public_time)):
    record.append(0)
    for i in range(len(report_time)):
        if j==0 and report_time.iloc[i]>public_time.iloc[j]:
            break
        elif j==0 and (public_time.iloc[j]-report_time.iloc[i]).days<150:
            record[j]=opaque_index_year.loc[name]["opaque"].iloc[i]
        if i==len(report_time)-1 and report_time.iloc[i]<public_time.iloc[j] and abs((public_time.iloc[j]-report_time.iloc[i]).days)<300:
            record[j]=opaque_index_year.loc[name]["opaque"].iloc[i]
            break
        elif report_time.iloc[i]<public_time.iloc[j] and report_time.iloc[i+1]>public_time.iloc[j] and abs((public_time.iloc[j]-report_time.iloc[i]).days)<300:
            record[j]=opaque_index_year.loc[name]["opaque"].iloc[i]
            break
op[QK=='K'] = record
cut_data.set_value(name,'opaque', op)
#cut_data.loc[name,'opaque'] = op

A


/shared_home/willie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,date,adjust_starttime,adjust_endtime,return,bad,url,QK,opaque
company,,,,,,,,
A,2000-03-15,2000-03-15,2000-06-12,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q,0.000000
A,2000-06-12,2000-06-12,2000-09-01,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q,0.000000
A,2000-09-01,2000-09-01,2001-01-17,0.893940,0.0,/Archives/edgar/data/1090872/00009120570004000...,Q,0.000000
A,2001-01-17,2001-01-17,2001-03-19,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K,0.000000
A,2001-03-19,2001-03-19,2001-06-14,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q,0.000000
A,2001-06-14,2001-06-14,2001-09-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,Q,0.000000
A,2001-09-14,2001-09-14,2002-01-22,1.462535,0.0,/Archives/edgar/data/1090872/00010958110150492...,Q,0.000000
A,2002-01-22,2002-01-22,2002-03-06,1.034088,0.0,/Archives/edgar/data/1090872/00008916180200018...,K,2.303592
A,2002-03-06,2002-03-06,2002-06-05,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...,Q,0.000000


In [110]:
len(cut_data.loc[name,'opaque'])

76

In [111]:
len(op)

76

In [112]:
for name in company_inboth:
    try:
        public_time=pd.to_datetime(cut_data.loc[name].set_index("QK").loc["K"]["date"])
        report_time=pd.to_datetime(opaque_index_year.loc[name]["time"])
        QK = cut_data.loc[name]['QK'].values
        op = cut_data.loc[name]['opaque'].values
        print(name)
        record=[]
        for j in range(len(public_time)):
            record.append(0)
            for i in range(len(report_time)):
                if j==0 and report_time.iloc[i]>public_time.iloc[j]:
                    break
                elif j==0 and (public_time.iloc[j]-report_time.iloc[i]).days<150:
                    record[j]=opaque_index_year.loc[name]["opaque"].iloc[i]
                if i==len(report_time)-1 and report_time.iloc[i]<public_time.iloc[j] and abs((public_time.iloc[j]-report_time.iloc[i]).days)<300:
                    record[j]=opaque_index_year.loc[name]["opaque"].iloc[i]
                    break
                elif report_time.iloc[i]<public_time.iloc[j] and report_time.iloc[i+1]>public_time.iloc[j] and abs((public_time.iloc[j]-report_time.iloc[i]).days)<300:
                    record[j]=opaque_index_year.loc[name]["opaque"].iloc[i]
                    break
        op[QK=='K'] = record
        cut_data.set_value(name,'opaque', op)
  #      cut_data.loc[name,'opaque'] = op
        print(name)
    except:
        continue

SUNN
SUNN
IAL


/shared_home/willie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


IAL
CTS
CTS
CRM
CRM
SGDE
SGDE
APEI
APEI
OMC
OMC
CRV
CRV
RESN
RESN
TCCO
TCCO
PCAR
PCAR
PM
PM
CPXX
CPXX
PSE
PSE
GEL
GEL
POOL
POOL
AMCS
AMCS
DOLE
DOLE
LNTH
LNTH
LRCX
LRCX
TA
TA
ARG
ARG
IOVA
WTT
WTT
SNPS
SNPS
ETS
ETS
BCGI
BCGI
ROCM
ROCM
DP
DP
ESAN
ESAN
TPTH
TPTH
CACI
CACI
FARM
FARM
FRP
FRP
AXE
AXE
PALM
PALM
MYE
MYE
WATT
WATT
HZO
HZO
LGTY
LGTY
SM
SM
ISLE
ISLE
CORE
PSAI
PSAI
HMII
HMII
MLSS
MLSS
KNDI
KNDI
TXI
TXI
NSC
NSC
SANM
SANM
ADLR
ADLR
DLHC
DLHC
AVP
AVP
HI
HI
DLPX
DLPX
MGRC
MGRC
ADBL
ADBL
BOYD
BOYD
CMPR
CMPR
UFPI
UFPI
OMN
OMN
NWRE
NWRE
COSN
COSN
FINL
FINL
LUB
LUB
EGAN
EGAN
DD
DD
THST
INFO
INFO
ADLI
ADLI
HUBB
HUBB
DOCX
DOCX
STON
STON
PNS
PNS
LFUS
LFUS
ESPR
ESPR
VSEC
VSEC
AMRS
AMRS
HQY
HQY
PRLS
PRLS
ARW
ARW
RMTR
RMTR
RAD
RAD
MSCC
MSCC
ALGT
ALGT
NKE
WYNN
WYNN
FLNT
ZVXI
ZVXI
WVE
WVE
SIGA
SIGA
PLXT
PLXT
ECL
ECL
KSWS
KSWS
ELMG
ELMG
AROC
AROC
SRR
SRR
TECU
TECU
HUSA
BERY
BERY
YUME
YUME
BURG
MVL
MVL
TLEO
TLEO
ZIGO
ZIGO
OPTR
OPTR
CAU
CAU
BSTC
BSTC
CAG
CAG
WYY
WYY
CATT
CATT
KAI
KAI
EDAC
EDAC
CSX
CS

BEBE
BEBE
MLAB
MLAB
DXR
DXR
CYTR
CYTR
SLNK
SLNK
CYTO
CYTO
PGNX
PGNX
SYNP
SYNP
N
N
ASTE
ASTE
BDC
BDC
LITE
LITE
DARA
DARA
CAB
CAB
LCRD
LCRD
SCLD
SCLD
AVD
AVD
DTLK
DTLK
INTT
INTT
GES
PDCO
PDCO
TMTA
TMTA
TFCO
TFCO
WEBX
WEBX
KODK
KODK
TBL
TBL
SPLS
SPLS
JOUT
JOUT
INTN
INTN
ODFL
ODFL
SBSE
SBSE
GPIC
GPIC
BWNG
BWNG
HIVE
HIVE
INTD
INTD
MMS
MMS
AINN
AINN
CMG
CMG
III
III
TUSK
TUSK
CIEN
CIEN
WMZ
WMZ
ATU
ATU
NCR
NCR
RVP
RVP
SLCA
SLCA
ICFI
ICFI
UAPH
UAPH
DM
DM
DVN
DVN
MRY
MRY
TLGD
TLGD
LOCO
LOCO
LXRX
LXRX
AWX
AWX
SCHN
SCHN
SGMA
SGMA
CPWM
CPWM
COLM
COLM
LPTX
LPTX
FDP
FDP
MANT
MANT
CSOD
CSOD
ENMC
ENMC
UFI
UFI
FB
FB
RNWK
RNWK
RHB
RHB
GNI
GNI
UTX
UTX
CRRC
CRRC
CXO
CXO
ANTP
ANTP
ISOL
ISOL
AMIN
AMIN
NTCT
NTCT
PBI
PBI
CLDA
CLDA
WWY
WWY
SHLX
SHLX
RT
RT
EPB
EPB
KSU
KSU
FANG
FANG
LZ
LZ
FRGI
FRGI
EMAN
EMAN
CMPX
CMPX
ACCO
ACCO
TTD
TTD
CECE
CECE
LCI
SGRP
SGRP
CAM
CAM
UEIC
UEIC
MTOR
MTOR
CAO
CAO
ARKR
ARKR
TRHC
TRHC
FORM
FORM
CTSO
CTSO
EVLV
EVLV
DIGL
DIGL
ICOC
ICOC
JMBA
JMBA
URGI
URGI
DEER
DEER
KFRC
KFRC
NSP
UUU
UU

CHTT
CHTT
ENCO
ENCO
HTSI
DANKY
DANKY
KWR
KWR
NTLS
NTLS
INFS
INFS
STRZ
STRZ
ULTA
ULTA
SALE
SALE
HAE
HAE
MTN
MTN
USLM
USLM
XOXO
XOXO
AXLE
LRN
LRN
TZIX
TZIX
ANDV
AAPL
AAPL
APA
APA
CPSH
CPSH
LANC
LANC
SPZN
SPZN
ULTI
ULTI
AXL
AXL
NMSCA
NMSCA
MLNM
MLNM
EPIQ
EPIQ
DLP
DLP
PSTG
PSTG
SMXC
SMXC
WMCO
WMCO
BR
BR
GTRC
GTRC
RS
RS
HURN
FLEX
FLEX
IREP
IREP
SMG
SMG
DAIO
DAIO
RP
RP
SBGI
SBGI
SYMC
SYMC
DOVR
DOVR
EQT
EQT
ATR
ATR
SGK
SGK
NGVC
NGVC
COP
COP
AKER
AKER
HELE
HELE
IRTC
IRTC
BLIN
BLIN
ROK
ROK
HIL
HIL
KTO
QUIX
QUIX
NPTN
KTCC
KTCC
FAME
FAME
BSTG
BSTG
RCL
RCL
XNCR
XNCR
JLMC
JLMC
RCRC
RCRC
GLRP
GLRP
TRMB
TRMB
SEH
SEH
INUV
INUV
INVE
INVE
RGLS
RGLS
ZSAN
ZSAN
NSYS
NSYS
ADXS
ADXS
IONS
IONS
QUMU
QUMU
MOLX
MOLX
AMCF
AMCF
GWRE
CHCI
CHCI
NUTR
NUTR
BCSI
BCSI
LIPD
LIPD
GBND
GBND
MPWR
MPWR
HWCC
HWCC
SHEN
SHEN
CLCT
CLCT
IOX
IOX
CNW
CNW
ZEUS
ZEUS
KND
KND
BDY
BDY
BIRT
BIRT
CEI
CEI
CONN
CONN
ADMP
ADMP
WMGI
WMGI
CPAK
CPAK
LEDR
LEDR
NFX
NFX
JVA
JVA
MCRI
MCRI
APMC
APMC
ISON
DNB
DNB
MITY
MITY
EXAC
EXAC
ALLN
ALLN
DKS
DKS

HTCH
MJN
MJN
RSYS
RSYS
LECO
LECO
PBG
PBG
BLBD
BLBD
AGCO
AGCO
GHCI
GHCI
DRTE
DRTE
CDNS
CDNS
LPNT
LPNT
FRAN
FRAN
NMHC
NMHC
MLHR
AMCO
AMCO
JH
JH
RBAK
RBAK
WSII
WSII
DDIC
DDIC
MSI
MSI
ORB
ORB
DWDP
APAC
APAC
DYNT
DYNT
RMP
RMP
AMN
AMN
PKI
PKI
PGLD
PGLD
AAI
AAI
STX
GSAT
GSAT
SAFM
SAFM
BKNG
BKNG
ECHO
ECHO
MACE
MACE
AOS
AOS
DERM
DERM
ARRO
ARRO
BOBE
BOBE
MDR
MDR
KEM
KEM
EXAS
EXAS
POPE
POPE
ELNK
ELNK
SPIR
SPIR
OBLN
OBLN
POSS
POSS
FLDM
FLDM
TUTR
TUTR
SOFO
SOFO
AHPI
AHPI
SGHL
SGHL
TLRA
SPNC
SPNC
CRVP
CRVP
VSTY
VSTY
MOSS
MOSS
BSX
BSX
VPI
VPI
SII
SII
EXEL
EXEL
GPX
GPX
TR
TR
QLTY
QLTY
AIPT
BSMD
BSMD
WLK
WLK
KTWO
KTWO
ABG
ABG
RICE
RICE
ISNS
ISNS
SPEX
SPEX
ALB
ALB
NTPL
NTPL
MEH
MEH
VOXW
VOXW
JNJ
JNJ
WAB
WAB
ALO
ALO
TWTR
TWTR
BZ
BZ
PBYI
PBYI
TMWD
TMWD
GETI
GETI
AIH
AIH
TTN
TTN
MON
MON
CNXR
CNXR
AAON
AAON
ESST
ESST
AIPC
AIPC
MCRS
MCRS
TBWC
TBWC
CPKI
CPKI
STMP
STMP
IBM
IBM
PENX
PENX
ANIP
ANIP
AAN
AAN
COVS
COVS
UHS
UHS
BVSN
BVSN
BSM
BSM
WIRE
WIRE
PYDS
PYDS
ESC
ESC
BALT
BALT
PZZA
PZZA
ONXS
ONXS
VLTR
VLTR
NCE

In [113]:
cut_data

,date,adjust_starttime,adjust_endtime,return,bad,url,QK,opaque
company,,,,,,,,
A,2000-03-15,2000-03-15,2000-06-12,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q,0.000000
A,2000-06-12,2000-06-12,2000-09-01,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q,0.000000
A,2000-09-01,2000-09-01,2001-01-17,0.893940,0.0,/Archives/edgar/data/1090872/00009120570004000...,Q,0.000000
A,2001-01-17,2001-01-17,2001-03-19,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K,0.000000
A,2001-03-19,2001-03-19,2001-06-14,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q,0.000000
A,2001-06-14,2001-06-14,2001-09-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,Q,0.000000
A,2001-09-14,2001-09-14,2002-01-22,1.462535,0.0,/Archives/edgar/data/1090872/00010958110150492...,Q,0.000000
A,2002-01-22,2002-01-22,2002-03-06,1.034088,0.0,/Archives/edgar/data/1090872/00008916180200018...,K,2.303592
A,2002-03-06,2002-03-06,2002-06-05,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...,Q,0.000000


In [114]:
for i in range(cut_data.shape[0]):
    if i<cut_data.shape[0]-2:
        if cut_data["opaque"].iloc[i] != 0:
            if ((cut_data["QK"].iloc[i+1:i+4] == "Q")/1).sum() == 3:
                cut_data.iloc[i+1:i+4,-1]=cut_data["opaque"].iloc[i]
            elif cut_data["opaque"].iloc[i] == cut_data["opaque"].iloc[i-1]:
                continue
            else:
                cut_data.iloc[i,-1]=0

In [115]:
cut_data

,date,adjust_starttime,adjust_endtime,return,bad,url,QK,opaque
company,,,,,,,,
A,2000-03-15,2000-03-15,2000-06-12,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q,0.000000
A,2000-06-12,2000-06-12,2000-09-01,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q,0.000000
A,2000-09-01,2000-09-01,2001-01-17,0.893940,0.0,/Archives/edgar/data/1090872/00009120570004000...,Q,0.000000
A,2001-01-17,2001-01-17,2001-03-19,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K,0.000000
A,2001-03-19,2001-03-19,2001-06-14,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q,0.000000
A,2001-06-14,2001-06-14,2001-09-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,Q,0.000000
A,2001-09-14,2001-09-14,2002-01-22,1.462535,0.0,/Archives/edgar/data/1090872/00010958110150492...,Q,0.000000
A,2002-01-22,2002-01-22,2002-03-06,1.034088,0.0,/Archives/edgar/data/1090872/00008916180200018...,K,2.303592
A,2002-03-06,2002-03-06,2002-06-05,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...,Q,2.303592


In [116]:
cut_data.to_csv("cutted_data_opaque.csv")

In [739]:
cut_data=pd.read_csv("cutted_data_opaque.csv",index_col=0)

In [117]:
cut_data["sec"]=0

In [59]:
cut_data.loc["AAPL"]

,date,adjust_datetime,return,bad,url,QK,opaque,sec
company,,,,,,,,
AAPL,1994-12-13,1994-12-13,0.993610,0.0,/Archives/edgar/data/320193/0000320193-94-0000...,K,0.000000,0
AAPL,1994-12-30,1994-12-30,0.973724,0.0,/Archives/edgar/data/320193/0000320193-95-0000...,Q,0.000000,0
AAPL,1995-03-31,1995-03-31,1.330258,0.0,/Archives/edgar/data/320193/0000320193-95-0000...,Q,0.000000,0
AAPL,1995-12-19,1995-12-19,0.918606,0.0,/Archives/edgar/data/320193/0000320193-95-0000...,K,0.000000,0
AAPL,1995-12-29,1995-12-29,0.803963,1.0,/Archives/edgar/data/320193/0000320193-96-0000...,Q,0.000000,0
AAPL,1996-03-29,1996-03-29,0.937823,0.0,/Archives/edgar/data/320193/0000320193-96-0000...,Q,0.000000,0
AAPL,1996-12-19,1996-12-19,0.664865,1.0,/Archives/edgar/data/320193/0000320193-96-0000...,K,0.000000,0
AAPL,1996-12-27,1996-12-27,0.791664,1.0,/Archives/edgar/data/320193/0000320193-97-0000...,Q,0.000000,0
AAPL,1997-03-28,1997-03-28,0.824501,1.0,/Archives/edgar/data/320193/0000320193-97-0000...,Q,0.000000,0


In [118]:
sector_industry=sector_industry.reset_index().set_index("index")

In [119]:
for name in list(set(cut_data.index)):
    try:
        cut_data.loc[name,["sec"]]=sector_industry.loc[name].iloc[0]
    except:
        cut_data.loc[name,["sec"]]="0"

In [120]:
cut_data.index

Index(['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       ...
       'SMRT', 'SMRT', 'SMRT', 'SMRT', 'SMRT', 'SMRT', 'SMRT', 'SMRT', 'SMRT',
       'SMRT'],
      dtype='object', name='company', length=283553)

In [121]:
cut_data

,date,adjust_starttime,adjust_endtime,return,bad,url,QK,opaque,sec
company,,,,,,,,,
A,2000-03-15,2000-03-15,2000-06-12,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q,0.000000,35
A,2000-06-12,2000-06-12,2000-09-01,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q,0.000000,35
A,2000-09-01,2000-09-01,2001-01-17,0.893940,0.0,/Archives/edgar/data/1090872/00009120570004000...,Q,0.000000,35
A,2001-01-17,2001-01-17,2001-03-19,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K,0.000000,35
A,2001-03-19,2001-03-19,2001-06-14,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q,0.000000,35
A,2001-06-14,2001-06-14,2001-09-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,Q,0.000000,35
A,2001-09-14,2001-09-14,2002-01-22,1.462535,0.0,/Archives/edgar/data/1090872/00010958110150492...,Q,0.000000,35
A,2002-01-22,2002-01-22,2002-03-06,1.034088,0.0,/Archives/edgar/data/1090872/00008916180200018...,K,2.303592,35
A,2002-03-06,2002-03-06,2002-06-05,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...,Q,2.303592,35


In [12]:
"""
industry_temp_memb=opaque_gic.loc[industry[0]]
industry_temp_memb=industry_temp_memb.dropna(how='any')
company_name=list(set(industry_temp_memb["tic"]))
industry_temp_memb=industry_temp_memb.set_index("tic")
industry_company=pd.DataFrame()
for i in company_name:
    try:
        temp_date=pd.to_datetime(industry_temp_memb.loc[i]["datadate"])
        shift_date=(temp_date-temp_date.shift(1)).dropna().iloc[-10:]
        last_date=pd.Series(shift_date.iloc[-10:])
        last_date=last_date.apply(lambda x: x.days)
        if len(last_date[last_date<120])==10 and len(temp_date)>12:
            industry_company=pd.concat([industry_company,industry_temp_memb.loc[i][["datadate","atq","revtq","xidocy","ibcy","oancfy","recchy","ppentq"]]])
    except:
        continue
filter_name=list(set(industry_company.index))
input_temp=pd.DataFrame(columns=["x1","x2","x3","x4","y","index_tic","index_time"])
for name in filter_name:
    caculate_temp=industry_company.loc[name].set_index("datadate")
    time_temp=pd.to_datetime(caculate_temp.index)
    caculate_temp.index=time_temp
    for j in range(1,len(time_temp)):
        if (time_temp[j]-time_temp[j-1]).days<120:
            if caculate_temp["atq"][j-1]>0:
                x1=1/caculate_temp["atq"][j-1]
                x2=(caculate_temp["revtq"][j]-caculate_temp["revtq"][j-1])/caculate_temp["atq"][j-1]
                x3=caculate_temp["ppentq"][j]/caculate_temp["atq"][j-1]
                if time_temp[j].month<5:
                    x4=industry_company["recchy"][j]/caculate_temp["atq"][j-1]
                else:
                    x4=(industry_company["recchy"][j]-industry_company["recchy"][j-1])/caculate_temp["atq"][j-1]
                y=(caculate_temp["ibcy"][j]-(caculate_temp["oancfy"][j]-caculate_temp["xidocy"][j]))/caculate_temp["atq"][j-1]
                input_temp=input_temp.append({"x1":x1,"x2":x2,"x3":x3,"x4":x4,"y":y,"index_tic":name,"index_time":time_temp[j]},ignore_index=True)
input_temp=input_temp.set_index(["index_tic","index_time"])
model = sm.OLS(input_temp["y"],input_temp[["x1","x2","x3"]])
results = model.fit()
params=results.params
dissacc=input_temp["y"]-(params[0]*input_temp["x1"]+params[1]*(input_temp["x2"]-input_temp["x4"])+params[2]*input_temp["x3"])
opaque_index=pd.DataFrame(columns=["time","opaque"])
for name in set(dissacc.index.get_level_values(0)):
    temp_stored=abs(dissacc.loc[name]).rolling(window=12).mean().dropna().reset_index()
    temp_stored.index=[name]*len(temp_stored)
    temp_stored.columns=["time","opaque"]
    opaque_index=pd.concat([opaque,temp_stored])
"""

In [13]:
"""
count=0
for i in company_name:
    try:
        temp_date=pd.to_datetime(industry_temp_memb.loc[i]["datadate"])
        shift_date=(temp_date-temp_date.shift(1)).dropna().iloc[-10:]
        last_date=pd.Series(shift_date.iloc[-10:])
        last_date=last_date.apply(lambda x: x.days)
        if len(last_date[last_date<120])==10:
            count =count+1
    except:
        continue
print(count)
print(len(company_name))
"""

150
226


In [ ]:
pd.read_csv("cut_data_all_QK.csv")

In [309]:
# opaque_collect=pd.DataFrame(columns=["Q1","Q2","Q3","Q4","Y"])
# for name in quarter_index:
#     try:
#         temp=opaque_index_year.loc[name]["time"]
#         temp2=opaque_index_quarter.loc[name].set_index("time").sort_index()
#         temp2.index=pd.to_datetime(temp2.index)
#         for i in range(1,len(temp)):
#             if len(temp2.loc[temp.iloc[i-1]:temp.iloc[i]].iloc[1:]) == 4:
#                 temp3=temp2.loc[temp.iloc[i-1]:temp.iloc[i]].iloc[1:]["opaque"].values.tolist()+[opaque_index_year.loc[name]["opaque"].iloc[i]]
#                 opaque_collect=pd.concat([opaque_collect,pd.DataFrame(temp3,index=["Q1","Q2","Q3","Q4","Y"]).T])
#     except:
#         continue
# opaque_collect=opaque_collect.reset_index().iloc[:,1:]
# model = sm.OLS(opaque_collect["Y"],opaque_collect[["Q1","Q2","Q3","Q4"]])
# results = model.fit()
# results.summary()

In [122]:
sector_industry=class_industry.copy()
sector_industry=sector_industry.reset_index().set_index("index")
for name in sector_industry.index:
    sector_industry.loc[name][0]=sector_industry.loc[name][0][:2]
sector_industry=sector_industry.reset_index().set_index("gic")

In [123]:
sector_industry=sector_industry.reset_index().set_index("index")

In [324]:
#marketcap_allcompany.to_csv("marketcap_allcompany.csv")

In [795]:
#cutted_data_opaque=pd.read_csv("cutted_data_opaque_removetext.csv")
#del cutted_data_opaque["text"]
#cutted_data_opaque.to_csv("cutted_data_opaque_removetext.csv")

In [126]:
cutted_data_opaque=cut_data.copy()

In [127]:
SP_return=pd.read_csv("SP_return.csv",index_col=0,parse_dates=True)
SP_price=(SP_return+1).cumprod()
#cutted_data_opaque=cutted_data_opaque.set_index("company")
cutted_data_opaque["SP_return"]=0

In [136]:
name="AAPL"
cutted_data_opaque.loc[name].shape[1]
time=pd.to_datetime(cutted_data_opaque.loc[name]["date"])
start_time=pd.to_datetime(cutted_data_opaque.loc[name]["adjust_starttime"])
end_time=pd.to_datetime(cutted_data_opaque.loc[name]["adjust_endtime"])
adj_return=[]
for j in range(len(time)-1):
    if (time.iloc[j+1]-time.iloc[j]).days<150:
        temp_price=SP_price.loc[time.iloc[j]+dt.timedelta(7):time.iloc[j+1]-dt.timedelta(7)]
        if len(temp_price)==0:
            adj_return.append(None)
        else:
            adj_return.append((temp_price.iloc[-1]/temp_price.iloc[0])[0])
    else :
        adj_return.append(None)
adj_return.append(None)
cutted_data_opaque.set_value(name,'SP_return', adj_return)

/shared_home/willie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,date,adjust_starttime,adjust_endtime,return,bad,url,QK,opaque,sec,SP_return
company,,,,,,,,,,
A,2000-03-15,2000-03-15,2000-06-12,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q,0.000000,35,0.0
A,2000-06-12,2000-06-12,2000-09-01,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q,0.000000,35,0.0
A,2000-09-01,2000-09-01,2001-01-17,0.893940,0.0,/Archives/edgar/data/1090872/00009120570004000...,Q,0.000000,35,0.0
A,2001-01-17,2001-01-17,2001-03-19,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K,0.000000,35,0.0
A,2001-03-19,2001-03-19,2001-06-14,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q,0.000000,35,0.0
A,2001-06-14,2001-06-14,2001-09-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,Q,0.000000,35,0.0
A,2001-09-14,2001-09-14,2002-01-22,1.462535,0.0,/Archives/edgar/data/1090872/00010958110150492...,Q,0.000000,35,0.0
A,2002-01-22,2002-01-22,2002-03-06,1.034088,0.0,/Archives/edgar/data/1090872/00008916180200018...,K,2.303592,35,0.0
A,2002-03-06,2002-03-06,2002-06-05,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...,Q,2.303592,35,0.0


In [137]:
cutted_data_opaque.loc["AAPL"]

,date,adjust_starttime,adjust_endtime,return,bad,url,QK,opaque,sec,SP_return
company,,,,,,,,,,
AAPL,1994-12-13,1994-12-13,1994-12-30,0.993610,0.0,/Archives/edgar/data/320193/0000320193-94-0000...,K,0.000000,45,1.005972
AAPL,1994-12-30,1994-12-30,1995-03-31,0.973724,0.0,/Archives/edgar/data/320193/0000320193-95-0000...,Q,0.000000,45,1.087459
AAPL,1995-03-31,1995-03-31,1995-06-30,1.330258,0.0,/Archives/edgar/data/320193/0000320193-95-0000...,Q,0.000000,45,NaN
AAPL,1995-12-19,1995-12-19,1996-02-13,0.918606,0.0,/Archives/edgar/data/320193/0000320193-95-0000...,K,0.000000,45,NaN
AAPL,1995-12-29,1995-12-29,1996-03-29,0.803963,1.0,/Archives/edgar/data/320193/0000320193-96-0000...,Q,0.000000,45,1.054970
AAPL,1996-03-29,1996-03-29,1996-06-28,0.937823,0.0,/Archives/edgar/data/320193/0000320193-96-0000...,Q,0.000000,45,NaN
AAPL,1996-12-19,1996-12-19,1997-02-11,0.664865,1.0,/Archives/edgar/data/320193/0000320193-96-0000...,K,0.000000,45,NaN
AAPL,1996-12-27,1996-12-27,1997-03-28,0.791664,1.0,/Archives/edgar/data/320193/0000320193-97-0000...,Q,0.000000,45,1.048222
AAPL,1997-03-28,1997-03-28,1997-06-27,0.824501,1.0,/Archives/edgar/data/320193/0000320193-97-0000...,Q,0.000000,45,NaN


In [138]:
for name in list(set(cutted_data_opaque.index)):
    try:
        cutted_data_opaque.loc[name].shape[1]
        time=pd.to_datetime(cutted_data_opaque.loc[name]["date"])
        start_time=pd.to_datetime(cutted_data_opaque.loc[name]["adjust_starttime"])
        end_time=pd.to_datetime(cutted_data_opaque.loc[name]["adjust_endtime"])
        adj_return=[]
        for j in range(len(time)-1):
            if (time.iloc[j+1]-time.iloc[j]).days<150:
                temp_price=SP_price.loc[start_time.iloc[j]+dt.timedelta(7):end_time.iloc[j]-dt.timedelta(7)]
                if len(temp_price)==0:
                    adj_return.append(None)
                else:
                    adj_return.append((temp_price.iloc[-1]/temp_price.iloc[0])[0])
            else :
                adj_return.append(None)
        adj_return.append(None)
        cutted_data_opaque.set_value(name,'SP_return', adj_return)
    except:
        cutted_data_opaque.loc[name,"SP_return"]=None

/shared_home/willie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [67]:
cutted_data_opaque["SP_return"].loc["AAPL"]

company
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
        ..
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
AAPL   NaN
Name: SP_return, Length: 83, dtype: float64

In [67]:
for name in list(set(cutted_data_opaque.index)):
    try:
        cutted_data_opaque.loc[name,"sec"]=sector_industry.loc[name].iloc[0]
    except:
        continue

In [139]:
cutted_data_opaque

,date,adjust_starttime,adjust_endtime,return,bad,url,QK,opaque,sec,SP_return
company,,,,,,,,,,
A,2000-03-15,2000-03-15,2000-06-12,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q,0.000000,35,0.978001
A,2000-06-12,2000-06-12,2000-09-01,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q,0.000000,35,1.013764
A,2000-09-01,2000-09-01,2001-01-17,0.893940,0.0,/Archives/edgar/data/1090872/00009120570004000...,Q,0.000000,35,0.878735
A,2001-01-17,2001-01-17,2001-03-19,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K,0.000000,35,0.865028
A,2001-03-19,2001-03-19,2001-06-14,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q,0.000000,35,1.107807
A,2001-06-14,2001-06-14,2001-09-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,Q,0.000000,35,0.877723
A,2001-09-14,2001-09-14,2002-01-22,1.462535,0.0,/Archives/edgar/data/1090872/00010958110150492...,Q,0.000000,35,1.186779
A,2002-01-22,2002-01-22,2002-03-06,1.034088,0.0,/Archives/edgar/data/1090872/00008916180200018...,K,2.303592,35,1.008404
A,2002-03-06,2002-03-06,2002-06-05,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...,Q,2.303592,35,0.925112


In [140]:
cutted_data_opaque.to_csv("cutted_data_opaque_sp.csv")

In [145]:
cutted_data_opaque.reset_index().set_index("sec").loc["0"]

,company,date,adjust_starttime,adjust_endtime,return,bad,url,QK,opaque,SP_return
sec,,,,,,,,,,
0,AA,2016-12-01,2016-12-01,2017-03-15,1.281695,0.0,/Archives/edgar/data/1675149/00011931251678270...,Q,0.0,1.051996
0,AA,2017-03-15,2017-03-15,2017-05-10,0.953943,0.0,/Archives/edgar/data/1675149/00011931251708386...,K,0.0,1.016896
0,AA,2017-05-10,2017-05-10,2017-08-03,1.130597,0.0,/Archives/edgar/data/1675149/00011931251716607...,Q,0.0,1.050225
0,AA,2017-08-03,2017-08-03,2017-10-27,1.241069,0.0,/Archives/edgar/data/1675149/00011931251724754...,Q,0.0,1.056188
0,AA,2017-10-27,2017-10-27,2018-02-26,0.997894,0.0,/Archives/edgar/data/1675149/00011931251732299...,Q,0.0,1.055794
0,AA,2018-02-26,2018-02-26,2018-05-09,1.227678,0.0,/Archives/edgar/data/1675149/00011931251805631...,K,0.0,0.968661
0,AA,2018-05-09,2018-05-09,2018-08-02,0.847249,0.0,/Archives/edgar/data/1675149/00011931251815767...,Q,0.0,1.042237
0,AA,2018-08-02,2018-08-02,2018-11-02,0.783753,0.0,/Archives/edgar/data/1675149/00011931251823676...,Q,0.0,0.931702
0,AA,2018-11-02,2018-11-02,2014-11-12,0.730623,0.0,/Archives/edgar/data/1675149/00011931251831725...,Q,0.0,NaN
